# Table of Contents
 <p><div class="lev1 toc-item"><a href="#IMPORTS" data-toc-modified-id="IMPORTS-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>IMPORTS</a></div><div class="lev1 toc-item"><a href="#Get-fixation-distributions" data-toc-modified-id="Get-fixation-distributions-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Get fixation distributions</a></div><div class="lev1 toc-item"><a href="#Switching-Tools" data-toc-modified-id="Switching-Tools-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Switching Tools</a></div><div class="lev1 toc-item"><a href="#Fitting" data-toc-modified-id="Fitting-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Fitting</a></div>

# IMPORTS

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import itertools
import os
import glob
import sys
# display matplotlib graphs 
%matplotlib inline

/scinet/gpc/Applications/anaconda3/4.2.0_hub/lib/python3.5/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')
/scinet/gpc/Applications/anaconda3/4.2.0_hub/lib/python3.5/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [3]:
sys.version

'3.5.2 |Anaconda custom (64-bit)| (default, Jul  2 2016, 17:53:06) \n[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]'

In [2]:
# Add path where custom modules are saved
sys.path.append('/Users/djw/Dropbox/PROGRAMMING/_NEURO/2017_MADE/aDDM_DJW/functions')
import utils_addm                         # for importing custom module

In [3]:
expdata_file_name = "data/made_v2/expdata.csv"
fixations_file_name = "data/made_v2/fixations.csv"

In [5]:
import glob
face_list = glob.glob('/Users/djw/Dropbox/PHD/CENDRI/Project/Code/LabSharedFolder/MADE01/CODE/v3_FractionalWeights/images/morphs/face/*.jpg')
house_list = glob.glob('/Users/djw/Dropbox/PHD/CENDRI/Project/Code/LabSharedFolder/MADE01/CODE/v3_FractionalWeights/images/morphs/house/*.jpg')

# create values to match to stimuli
values = np.arange(-1.,1.01,0.02)

# Get rid of extra digits
for i in range(len(values)):
    values[i] = round(values[i], 2)

# Create Data Frames
house_df = pd.DataFrame(
    {'value': values,
     'exemplar': house_list
    })

face_df = pd.DataFrame(
    {'value': values,
     'exemplar': face_list
    })

In [16]:
face_df.exemplar[0]

'/Users/djw/Dropbox/PHD/CENDRI/Project/Code/LabSharedFolder/MADE01/CODE/v3_FractionalWeights/images/morphs/face/faceMorph000.jpg'

In [7]:
# create values to match to stimuli
values = np.arange(-1.,1.01,0.02)

# Get rid of extra digits
for i in range(len(values)):
    values[i] = round(values[i], 2)

# Create Data Frames
house_df = pd.DataFrame(
    {'value': values,
     'exemplar': house_list
    })

face_df = pd.DataFrame(
    {'value': values,
     'exemplar': face_list
    })


# Get fixation distributions

# Switching Tools

# Fitting

In [4]:
path = "/Users/djw/Dropbox/PROGRAMMING/_NEURO/2017_MADE/aDDM_DJW/outputs/2017-11-16/" # path to saved sim shelve file

def join_all_sims(path):
    """
    """
    
    all_files = glob.glob(os.path.join(path, "*.csv"))                        # advisable to use os.path.join as this makes concatenation OS independent

    dfs = {}  # create dict to hold dataframes
    x = 0

    for f in all_files:
        dfs[x] = pd.read_csv(f)
        dfs[x] = dfs[x].drop(dfs[x].columns[[0]], axis=1)  # get rid of unnamed column
        x+=1
    return dfs

In [5]:
dfs = join_all_sims(path)

In [8]:
# Get left and right values from data
df = pd.DataFrame.from_csv(expdata_file_name, header=0, sep=",", index_col=None)

item_left_val = np.unique(df.item_left_val)
item_right_val = np.unique(df.item_right_val)

values = list(itertools.product(item_left_val, item_right_val))
values_list = np.around(values,decimals=2)   # currently produces 81 combos

nonDec = 0.8

# Function to calculate rt Distribution for simulated parameter combos
rt_dist = rtDistFunc(nonDec, values_list, dfs)

# Save RT Dist (pickle or shelve)

# Figure out how to shelve correctly (maybe just can't interrupt?)
# See if can extract params
# Fit 1/2 trials
# Test Fit

In [11]:
x = rt_dist['0.0514285714286_0.278571428571_0.325']['1.63_0.42'][:,0]
y_accept = rt_dist['0.0514285714286_0.278571428571_0.325']['1.63_0.42'][:,1]
y_reject = rt_dist['0.01_0.05_0.1']['1.63_0.42'][:,2]

In [18]:
sys.getsizeof(dfs)

9320

In [17]:
sys.getsizeof(rt_dist)

9320

In [ ]:
plt.plot(x,y_accept)


In [7]:
def rtDistFunc(nonDec, values_list, dfs):
    
    # Create upper level dict to hold param combos
    rt_dist = {}
    
    # Create bins for RT distribution
    # ?? Add in NDT or not??
    bins = np.arange(nonDec,10.1,.1)             # start, stop, step (used for hist)
    binz = np.delete(bins,-1)                    # delete the last value so bin number = count number
    
    # Make list of all value combos (or supply as argument)
    values_list_keys = []
    for i in range(len(values_list)):
        values_list_keys.append(str(values_list[i,0]) + '_' + str(values_list[i,1]))
    
    for d in dfs:

        # Name for outer dict based on the valWeight and upperBound with an integer (as STRING) leading
        param_combos = str(dfs[d].scaling[0]) + '_' + str(dfs[d].upper_boundary[0]) + '_' + str(dfs[d].theta[0])
        # Create nested dict to hold values
        rt_dist[param_combos] = {}
                
        # create subsets of RT for each value combo
        for i in range(len(values_list)):
            data = dfs[d][(dfs[d].val_face == values_list[i,0]) & (dfs[d].val_house == values_list[i,1])]
            data0 = data[data.resp == -1]             # select reject responses
            data1 = data[data.resp == 1]              # select accept responses

            # Create RT distrib (counts/bin)
            count0, bins = np.histogram(data0.rt, bins)  # unpack the reject counts in each bin
            count1, bins = np.histogram(data1.rt, bins)  # unpack the accept counts in each bin

            length = float(sum(count0) + sum(count1)) # number of non NaN values

            # initialize array to hold Distribs
            distrib = np.ndarray((len(count0), 3))
            distrib[:,0] = binz                       # bin values from START of bin
            distrib[:,1] = count0 /length              # reject
            distrib[:,2] = count1 /length              # accept

                        # select the rows with given drift  # remove all columns except rt and resp
            value_key = values_list_keys[i]
            rt_dist[param_combos][value_key] = distrib
    
    return rt_dist
            

In [106]:
dfsActualParticipant = pd.read_csv('/Users/djw/Dropbox/PROGRAMMING/PythonLearning/DDM/examples/cleaned_oneImage_Mult.csv')

dfsActualParticipant.drop(dfsActualParticipant.columns[[3]], axis=1, inplace=True)
dfsActualParticipant.rename(columns={"summedVal": 'value',
                                     'acceptReject': 'resp'}, inplace=True)

dfsActualParticipant.loc[dfsActualParticipant['resp'] == 0, 'resp'] = -1.


In [107]:
dfsActualParticipant.drop(dfsActualParticipant.columns[4:], axis=1, inplace=True)

In [108]:
dfsActualParticipant.head()

,subj_idx,value,rt,resp
0,1,3.04,8.883308,1.0
1,1,-0.14,4.550172,1.0
2,1,0.28,3.716686,1.0
3,1,-0.52,4.516826,-1.0
4,1,0.04,2.966837,1.0


In [110]:
participants = pd.read_csv(expdata_file_name)

In [111]:
participants.head()
# Divide RT by 1000
# rename item left val and right val to face and house


,parcode,trial,rt,choice,item_left,item_right,item_left_val,item_right_val
0,1,1,8883.308377,1,1.32,1.72,1.63,1.69
1,1,2,4550.171639,1,0.38,-0.52,0.41,-0.39
2,1,3,3716.686491,1,0.96,-0.68,0.89,-0.66
3,1,4,4516.825782,-1,-0.94,0.42,-0.89,0.42
4,1,5,2966.836687,1,-0.24,0.28,-0.37,0.42


In [ ]:
# Value to replace zero values...
epsilon = np.finfo(float).eps

# TURN INTO FUNCTION
for key in np.sort(rtDist.keys()):                                   # go through all keys of rtDist (which are the paramCombos)
    rtDistList = []    # create list to store data

    valWeight = float(key.split('_')[0])                             # splitting and choosing first element (weight)
    for x in range(len(dfsActualParticipant)):
        driftVal = round((dfsActualParticipant.value[x] * valWeight), 8)    # calculating driftVal based on value and weight
        
        row = int((dfsActualParticipant.rt[x]-nonDec)/.1)                # increment by 1 for every increase in .1 seconds (taking into account ndt)
        if (dfsActualParticipant.resp[x] == -1):                     # if reject
            rtDistList.append(rtDist[key][driftVal][row,1])
            #dfsSimParticipant[s][key][x] = rtDist[key][driftVal][row,1]

        else:                                                        # if accept
            rtDistList.append(rtDist[key][driftVal][row,2])
            #dfsSimParticipant[s][key][x] = rtDist[key][driftVal][row,2]

    dfsActualParticipant[key] = rtDistList                           # Create column from list
    dfsActualParticipant = dfsActualParticipant.replace(to_replace=0, value=epsilon)     # Remove values of 0 and replace with Epsilon

#dfsSimParticipant[s] = np.log(dfsSimParticipant[s].iloc[:,4:]) * -1  # Convert to natural log and Remove unnecessary columns
dfsActualParticipant = np.log(dfsActualParticipant.iloc[:,4:]) * -1  # Convert to natural log and Remove unnecessary columns

In [ ]:
subject = {}

for x in range(len(dfsSimParticipant)):
    subject[x] = pd.DataFrame(dfsSimParticipant[x].sum(axis=0))
    
    weight = []
    boundary = []
    
    for m in range(len(subject[x].index)):
        weight.append(float(subject[x].index[m].split('_')[0]))      # create list of weight values
        boundary.append(float(subject[x].index[m].split('_')[1]))    # create list of boundary values

    subject[x]['weight'] = weight                                       # create column with weight values
    subject[x]['boundary'] = boundary                                   # create column with boundary values

subject[0].head()